<a href="https://colab.research.google.com/github/Elwing-Chou/tibaml0922/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/tibaml0922/raw/main/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/tibaml0922/raw/main/test.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7f52a86a0b10>)

In [4]:
import pandas as pd
data = pd.read_csv("train.csv", encoding="utf-8")
predict = pd.read_csv("test.csv", encoding="utf-8")

In [5]:
predict

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [ ]:
df = pd.concat([data, predict],
                axis=0,
                ignore_index=True)
df = df.drop(["PassengerId", "Survived"],
             axis=1)
df

In [ ]:
def cabin(c):
    if not pd.isna(c):
        return c[0]
    else:
        return None
df["Cabin"] = df["Cabin"].apply(cabin)
df

In [25]:
dic = df["Ticket"].value_counts()
df["Ticket"] = df["Ticket"].replace(dic)

In [30]:
def name(n):
    n = n.split(",")[-1].split(".")[0]
    n = n.strip()
    return n
df["Name"] = df["Name"].apply(name)
# name("Kelly, Mr. James")

In [32]:
nasum = df.isna().sum()
# 篩選: Series[跟你資料筆數一樣多的T/F]
nasum[nasum > 0].sort_values(ascending=False)

Cabin       1014
Age          263
Embarked       2
Fare           1
dtype: int64

In [35]:
med = df.median().drop(["Pclass"])
df = df.fillna(med)
nasum = df.isna().sum()
# 篩選: Series[跟你資料筆數一樣多的T/F]
nasum[nasum > 0].sort_values(ascending=False)

Cabin       1014
Embarked       2
dtype: int64

In [38]:
most = df["Embarked"].value_counts().idxmax()
df["Embarked"] = df["Embarked"].fillna(most)
nasum = df.isna().sum()
# 篩選: Series[跟你資料筆數一樣多的T/F]
nasum[nasum > 0].sort_values(ascending=False)

Cabin    1014
dtype: int64

In [48]:
nc = df["Name"].value_counts()
whitelist = nc[nc > 50].index
def namemid(n):
    if n not in whitelist:
        return None
    else:
        return n
df["Name"] = df["Name"].apply(namemid)

In [ ]:
df = pd.get_dummies(df)
df = pd.get_dummies(df, columns=["Pclass"])
df

In [52]:
df["Family"] = df["SibSp"] + df["Parch"]
df

,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Family
0,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1
1,38.0,1,0,2,71.2833,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1
2,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1
4,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1305,39.0,0,0,3,108.9000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0
1306,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1307,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
